<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Finanzas Cuantitativas. 

**Nombres:** Jesús Álvarez, Ariadna Galindo, Romina Gutierrez y Rodolfo Slay.

**Fecha:** 9 de septiembre del 2020.

**Expediente** : 714552, 718272, 718415 y 715214.


# Tarea 3.1

Revisit the example from previous class:

"*Buy a start up stock that doesn't pay dividends, estimate the Expected Results using Corporate Finance Theory the X factor Stochastic*"Asume a normal distribution. 

    a) Analytical method 
    b) Simulate in excel/python to test accuracy


### a) Método analítico

Recordemos la fórmula de normal ya que hemos asumido esta distribución. 
$$f(x)=\frac{1}{\sigma \sqrt{2\pi}}e^{\frac{(x-\mu)^2}{2\sigma^2}}$$
Volveremos a utilizar autozone (AZO)

*Esperanza matemática*

$$ r =  \alpha + \beta (m - rf) + rf $$

$$ E[r] = E[\alpha + \beta (m - rf) + rf] $$
$$ E[r] = E[\alpha + \beta (m - rf) + rf] $$
$$ E[r] = E[\alpha] + E[\beta (m - rf)] + E[rf] $$
$$ E[r] = E[\alpha] + \beta E[m - rf] + E[rf] $$
$$ E[r] = E[\alpha] + \beta E[m] - \beta E[rf] + E[rf] $$
$$ E[r] = E[\alpha] + \beta E[m] - \beta * rf + rf $$

$$ E[r] = 0.12 - 0.8113 (359.0910) + 0.8113 * 0.03 + 0.03 $$

$$ E[r] =  -291.1561 $$


$$ r =  \alpha + \beta (m - rf) + rf $$

$$ Var[r] = Var [\alpha + \beta (m - rf) + rf] $$
$$ Var[r] = Var[\alpha] + Var[\beta (m - rf)] + Var[rf] + 2 *Cov(\alpha,m)$$
$$ Var[r] = Var[\alpha] + \beta^2 Var[m - rf] + Var[rf] + 2 *Cov(\alpha,m)$$
$$ Var[r] = Var[\alpha] + \beta^2 Var[m] - \beta^2 Var[rf] + Var[rf] + 2 *Cov(\alpha,m)$$
$$ Var[r] = Var[\alpha] + \beta^2 Var[m] - \beta^2 * 0 + 0 + 2 *Cov(\alpha,m)$$
$$ Var[r] = Var[\alpha] + \beta^2 Var[m] + 2 *Cov(\alpha,m)$$
$$ Var[r] = Var[\alpha] + \beta^2 Var[m] + 2 * \sqrt{Var[\alpha]*Var[m]}*Corr[\alpha,m]$$

$$ Var[r] = 17973.4348 + (-0.8113)^2 49126.3344 + 2 * \sqrt{17973.4348*49126.3344}*0 $$

$$ Var[r] = 50310.7659 $$

### b) Simulación

In [1]:
#importamos librerias 
import scipy.stats as st
import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web 
import warnings
import numpy as np

In [4]:
#funcion para ajustar distribucion 
def best_fit_distribution(data, bins=1000, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0
    
    DISTRIBUTIONS = [        
         st.gennorm,st.genexpon,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
          st.nct,st.norm,st.powerlognorm, st.uniform
         ,st.genexpon,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
          st.norm,st.powerlognorm, st.uniform
    ]
# Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf
    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)

# Función para descargar precios de cierre ajustados:
def get_adj_closes(tickers, start_date=None, end_date=None):
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    closes = closes['Adj Close']
    closes.sort_index(inplace=True)
    return closes
# Información
names = ["AZO", '^GSPC']
start= '2019-09-09'

In [22]:
closes= get_adj_closes(tickers =names,
                      start_date = start)
ret = closes.pct_change().dropna()
azo=closes.iloc[:,0]
gspc=closes.iloc[:,1]

ret_azo = ret.iloc[:,0]
ret_gspc = ret.iloc[:,1]

Beta = gspc.corr(azo)
print('Beta: ' + str(Beta))
best_fit_azo, azo_params = best_fit_distribution(azo, 200)
print('AZO ~ N(' + str(azo_params[0]*252) + ', ' + str(azo_params[1]*252**0.5) + ')')
best_fit_gspc, gspc_params = best_fit_distribution(gspc, 200)
print('GSPC ~ N(' + str(gspc_params[0]*252) + ', ' + str(gspc_params[1]*252**0.5) + ')')

print('\n\n')

Beta: 0.8113251443694939
AZO ~ N(242.35415277225084, 17973.43483142189)
GSPC ~ N(359.09103567582434, 49126.33444843748)





# Tarea 3.2
Using the properties of random variables and knowing the PDF of the sales product of each year answer analytically:

    a) What are the expected product sales for year 1 and year 2?
    b) What is the expected revenue in PV for the life of the product? What is the VAR?
    c) What is the expected NPV and VAR for NPV?
    d) Estimate the PDF of NVP and IRR answer analytically:
        1. What the probability that the IRR is over the Risk free rate?
        2. What the probability that the IRR is over 35%?
        3. What the probability that the project value is over 2M?
        4. What's the probability that the IRR is between 10% and 20%?
        5. Do you get same responses as in Homework 1.1?